In [ ]:
string = 'BBNR(df, k = k)'
string='sink({}, type=message)'.format(string)
string

In [13]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import os
#os.chdir('/fast/work/groups/ag_kircher/ReMM/MA_Lusi/Snakemake')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [14]:
import pandas as pd
import numpy as np
from random import sample 
import sys 
sys.path.insert(0, 'scripts/')
from utils import *

In [6]:
modelsR = [ 
            'AENN(df)' 
            ,'BBNR(df, k = k)'
           ,'C45robustFilter(df)'
           ,'C45votingFilter(df, nfolds = nfolds, consensus = FALSE)'
           ,'C45iteratedVotingFilter(df, nfolds = nfolds, consensus = FALSE)'
           ,'CNN(df)'
           ,'CVCF(df, nfolds = nfolds, consensus = FALSE)'
           ,'DROP1(df, k = 1)'
           ,'DROP2(df, k = 2)'
           ,'DROP3(df, k = 3)'
           ,'dynamicCF(df, nfolds = nfolds, consensus = FALSE, m = 3)'
           ,'edgeBoostFilter(df, m = 15, percent = 0.05, threshold = 0.2)'
           ,'EF(df, nfolds = nfolds, consensus = TRUE)'
           ,'ENG(df, graph = "RNG")'
           ,'EWF(df, threshold = 0.25, noiseAction = "remove")'
           ,'GE(df, k = k, kk = ceiling(k/2))'
           ,'HARF(df, nfolds = nfolds, agreementLevel = 0.7, ntrees = 500)'
           ,'hybridRepairFilter(df, consensus = FALSE, noiseAction = "remove")'
           ,'INFFC(df, consensus = FALSE, p = 0.01, s = 1, k = k, threshold = 0)'
           ,'IPF(df, nfolds = nfolds, consensus = FALSE, p = 0.01, s = 3, y = 0.5)'
           ,'ModeFilter(df, type = "classical", noiseAction = "repair", epsilon = 0.05, maxIter = 100, alpha = 1, beta = 1)'
           ,'ORBoostFilter(df, N = 20, d = 11, Naux = max(20, N), useDecisionStump = FALSE)'
           ,'PF(df, nfolds = nfolds, consensus = FALSE, p = 0.01, s = 3, y = 0.5, theta = 0.7)'
           ,'PRISM(df)'
           ,'RNN(df)'
           ,'saturationFilter(df, noiseThreshold = noiseThreshold)'
           ,'consensusSF(df, nfolds = nfolds, consensusLevel = nfolds - 1, noiseThreshold = noiseThreshold)'
           ,'classifSF(df, nfolds = nfolds, noiseThreshold = noiseThreshold)'
           ,'TomekLinks(df)'
]



In [7]:
mo = pd.Series(modelsR).str.split('(',expand=True)[0]
#mo

In [8]:
li = ['ORBoostFilter', 'IPF', 'CNN', 'HARF', 'C45robustFilter', 'C45votingFilter', 'BBNR', 'CVCF', 
 'C45iteratedVotingFilter']
li = pd.Series(li)
lii = mo[~mo.isin(li)]

In [9]:
print(lii.to_list())

['AENN', 'DROP1', 'DROP2', 'DROP3', 'dynamicCF', 'edgeBoostFilter', 'EF', 'ENG', 'EWF', 'GE', 'hybridRepairFilter', 'INFFC', 'ModeFilter', 'PF', 'PRISM', 'RNN', 'saturationFilter', 'consensusSF', 'classifSF', 'TomekLinks']


In [10]:
len(modelsR)
# ERRORS:
# saturationFilter, consensusSF, classifSF  missing value where TRUE/FALSE needed

29

In [12]:
lii = []
for i in li:
    df = pd.read_csv(i, sep = '\t',header = None)
    if (df[1]==0).sum()<2:
        lii.append(i)
      #  print(i)
       # display(df)

FileNotFoundError: [Errno 2] No such file or directory: 'ORBoostFilter'

In [22]:
print(pd.Series(lii).str.split('_',expand=True)[4].to_list())

['ORBoostFilter', 'AENN', 'IPF', 'CNN', 'HARF', 'C45robustFilter', 'C45votingFilter', 'BBNR', 'CVCF', 'RNN', 'C45iteratedVotingFilter']


In [ ]:
['ORBoostFilter', 'IPF', 'CNN', 'HARF', 'C45robustFilter', 'C45votingFilter', 'BBNR', 'CVCF', 
 'C45iteratedVotingFilter']


In [ ]:
["AENN","CVCF", "EF","GE", "HARF", "hybridRepairFilter", "IPF", "ORBoostFilter", "RNN"]

In [4]:
from glob import glob 
li = glob('RModels/*.tmp')
mo = pd.Series(li).str.split('_',expand=True)[4]

In [6]:
al = pd.Series(modelsR).str.split('(',expand=True)[0]

In [7]:
m = pd.Series(["AENN","CVCF", "edgeBoostFilter", "EF","GE", "HARF", "hybridRepairFilter", "IPF", 
               "ORBoostFilter", "RNN"])
al[~al.isin(mo)]

7                DROP1
8                DROP2
9                DROP3
13                 ENG
14                 EWF
18               INFFC
20          ModeFilter
25    saturationFilter
26         consensusSF
27           classifSF
Name: 0, dtype: object

In [16]:
import sys
import numpy as np
import random
import pandas as pd
import warnings 
from time import time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import traceback


warnings.filterwarnings('ignore')
sys.path.insert(0, 'scripts/')

from runRFilter import *
from cleanLabFilter import CleanLab
from filtersScikiClean import filtersScikiClean
from utils import *
from addNoiseScikit import *

#path = str(snakemake.output)
path = 'temp/Encode_0.1_100_Asym_KDN_Python.tmp'
path="temp/Adult_0.3_10000_Sym_KDN_Python.tmp"

#st = 'temp/Magic_0.1_100_Asym_edgeBoostFilter_R.tmp'
st  = path.split('/')[1].split('.tmp')[0].split('_')
name = st[0]


#repeats = int(snakemake.params.repeats)
repeats = 1

df_raw = pd.read_csv('datasets/' + name + '.csv.gz', sep = '\t',compression='zip',
                index_col=None)

In [17]:


noiseLevel = float(st[1])
datasetSize = int(st[2])
noiseType = st[3]
model = st[4]
imp = st[5]

df = df_raw

dfMeansCV = pd.DataFrame()
dfMeansScores = pd.DataFrame()   
for r in range(repeats):
    ID = [name, model, noiseLevel, noiseType,datasetSize,r]

    X, y, noisyLabels = getData(df,name, noiseType, noiseLevel, datasetSize)

    noiseInd = y[y!=noisyLabels].index
    dR = pd.DataFrame(np.vstack([X.T, noisyLabels.tolist()]).T)



model
datasetSize

10000

In [24]:
dRAll = pd.DataFrame(np.vstack([X.T, noisyLabels.tolist(),y.to_list()]).T)
(dRAll[107]!=dRAll[108]).sum()

3000

In [25]:
dRAll.to_csv('test/Adult.csv',sep = '\t',header = None, index = None)

In [27]:

df = df_raw

dfMeansCV = pd.DataFrame()
dfMeansScores = pd.DataFrame()   
for r in range(repeats):
    ID = [name, model, noiseLevel, noiseType,datasetSize,r]

    X, y, noisyLabels = getData(df,name, noiseType, noiseLevel, datasetSize)

    noiseInd = y[y!=noisyLabels].index

    dR = pd.DataFrame(np.vstack([X.T, noisyLabels.tolist()]).T)

    t0 = time()
    if imp=='Python':
        filteredNoiseInd = filtersScikiClean(X,y,noisyLabels, t = 0.5,n = noiseLevel, model = [model])
    if imp=='CleanLab':
        filteredNoiseInd  = CleanLab(X,y,noisyLabels)
    if imp=='R':
        filteredNoiseInd = getRModel(dR, y, noisyLabels,model = [model])

    cv, scores = confusionMatrixScikit(y,noiseInd, filteredNoiseInd)
    cv.index = [model]
    scores.index = [model]

    t1 = time()
    totalTime = t1-t0
    cv['Execution Time'] = totalTime
    temp = pd.DataFrame([len(y), noiseInd.to_list(), filteredNoiseInd.to_list(),t1],
                 index = ['N', 'NoiseInd', 'FoundNoiseInd','Time']).T
    temp.index=[model]
    cv = cv.join(scores).join(temp)
    cv.index = [str(ID)]
    # print(totalTime)
    dfMeansCV = dfMeansCV.append(cv)

cv

['PRISM']


R[write to console]: Caught an error!

R[write to console]: missing value where TRUE/FALSE needed


,True Positive,False Positive,True Negative,False Negative,Execution Time,Precision,Recall,F-score,N,NoiseInd,FoundNoiseInd,Time
"['ClinVarReal', 'PRISM', 0.1, 'Real', 1000, 0]",NaN,NaN,NaN,NaN,6.4021,0.0000,0.0000,0.0000,1000,"[903, 924, 900, 941, 997, 901, 962, 975, 915, ...",[failed!],1644063246.1544


In [28]:
cv

,True Positive,False Positive,True Negative,False Negative,Execution Time,Precision,Recall,F-score,N,NoiseInd,FoundNoiseInd,Time
"['ClinVarReal', 'PRISM', 0.1, 'Real', 1000, 0]",NaN,NaN,NaN,NaN,6.4021,0.0000,0.0000,0.0000,1000,"[903, 924, 900, 941, 997, 901, 962, 975, 915, ...",[failed!],1644063246.1544


In [27]:
cv

,True Positive,False Positive,True Negative,False Negative,Execution Time,Precision,Recall,F-score,N,NoiseInd,FoundNoiseInd,Time
"['ClinVarReal', 'ModeFilter', 0.3, 'Real', 100, 0]",NaN,NaN,NaN,NaN,83.359408,0.0,0.0,0.0,100,"[76, 85, 95, 92, 93, 98, 99, 86, 83, 75, 73, 7...",[<rpy2.rinterface_lib.sexp.NULLType object at ...,1643893516.473957


In [49]:
dR[dR[131].reset_index(drop = True)==y.reset_index(drop = True)]

,0,1,2,3,4,5,6,7,8,9,...,122,123,124,125,126,127,128,129,130,131
0,0.470,0.040,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.358,0.000,0.0,0.0,0.0,1933.0,0.329,1839.0,0.395,613.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.596,0.067,0.0,0.0,0.0,3040.0,0.393,2854.0,0.497,952.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.695,0.173,0.0,0.0,0.0,10.0,0.002,1.0,0.001,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
8,0.351,0.040,0.0,0.0,0.0,431.0,0.076,168.0,0.045,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,0.318,0.000,0.0,0.0,0.0,1179.0,0.271,993.0,0.247,331.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9992,0.497,0.013,0.0,0.0,0.0,26744.0,0.964,26142.0,0.990,8714.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9993,0.596,0.013,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9998,0.397,0.000,0.0,0.0,0.0,2952.0,0.631,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from memory_profiler import memory_usage
from memory_profiler import memory_usage

from line_profiler import LineProfiler


In [ ]:

lp = LineProfiler()
lp_wrapper = lp(getRModel)
lp_wrapper(dR, y, noisyLabels,model = [model])
lp.print_stats()

In [ ]:
import psutil

print(psutil.cpu_percent())

In [7]:
import sys
import numpy as np
import random
import pandas as pd
import warnings 
from time import time
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import traceback


warnings.filterwarnings('ignore')
sys.path.insert(0, 'scripts/')

from runRFilter import *
from cleanLabFilter import CleanLab
from filtersScikiClean import filtersScikiClean
from utils import *
from addNoiseScikit import *


#path = str(snakemake.output)
path = 'temp/Adult_0.7_100_Asym_hybridRepairFilter_R.tmp'
#st = 'temp/Magic_0.1_100_Asym_edgeBoostFilter_R.tmp'
# [Adult, hybridRepairFilter, 0.7, Sym, 1000, 0]
st  = path.split('/')[1].split('.tmp')[0].split('_')
name = st[0]
noiseLevel = float(st[1])
datasetSize = int(st[2])
noiseType = st[3]
model = st[4]
imp = st[5]

#repeats = int(snakemake.params.repeats)
repeats = 1
df = pd.read_csv('datasets/' + name + '.csv.gz', sep = '\t',compression='zip',
                index_col=None, nrows = 1000)

ID = [name, model, noiseLevel, noiseType,datasetSize]

dfMeansCV = pd.DataFrame()
try:
    for r in range(repeats):

        ID = [name, model, noiseLevel, noiseType,datasetSize,r]

        X, y, noisyLabels = getData(df,name, noiseType, noiseLevel, datasetSize)

        noiseInd = y[y!=noisyLabels].index

        dR = pd.DataFrame(np.vstack([X.T, noisyLabels.tolist()]).T)

        t0 = time()
        if imp=='Python':
            filteredNoiseInd = filtersScikiClean(X,y,noisyLabels, t = 0.5,n = noiseLevel, model = [model])
        if imp=='CleanLab':
            filteredNoiseInd  = CleanLab(X,y,noisyLabels)
        if imp=='R':
            filteredNoiseInd = getRModel(dR, y, noisyLabels,model = [model])
        
        #print(filteredNoiseInd)
        cv, scores = confusionMatrixScikit(y,noiseInd, filteredNoiseInd)
        cv.index = [model]
        scores.index = [model]

        t1 = time()
        totalTime = t1-t0
        cv['Execution Time'] = totalTime
        
        temp = pd.DataFrame([len(y), noiseInd.to_list(), filteredNoiseInd.to_list(),t1],
                 index = ['N', 'NoiseInd', 'FoundNoiseInd','Time']).T
        temp.index=[model]
        cv = cv.join(scores).join(temp)
        cv.index = [str(ID)]

       # print(totalTime)
        dfMeansCV = dfMeansCV.append(cv)

    
except Exception as e: 
    print('CAUGHT AN ERROR IN ', str(ID))
    print(e)
    log = traceback.format_exc()
    #log   = ''.join(traceback.format_tb(e.__traceback__))

    pd.Series(log).to_csv('logs/'+str(ID)+'.log', sep = '\t',header = False)
    dfMeansCV = pd.DataFrame()
    dfMeansCV[str(ID)] = ['failed']

    
#dfMeansCV.iloc[:,:9].to_csv('output/'+name+'_'+imp+'.csv', sep = '\t',header = False, mode='a')
#dfMeansCV.iloc[:,:9].to_csv(path, sep = '\t',header = False)

#dfMeansCV.to_csv('output/'+name+'_'+imp+'_Extended.csv', sep = '\t',header = False, mode='a')
#dfMeansCV.to_csv(path+'Extended', sep = '\t',header = False)

#cvsMean.T.to_csv('temp/'+name+'.csv', sep = '\t')


['hybridRepairFilter']
0      2
1      3
2      4
3     10
4     16
5     17
6     21
7     23
8     24
9     26
10    30
11    34
12    37
13    38
14    40
15    45
16    47
17    49
18    52
19    53
20    54
21    58
22    60
23    69
24    71
25    73
26    75
27    80
28    83
29    88
30    96
31    98
dtype: int64


In [8]:
cv


,True Positive,False Positive,True Negative,False Negative,Execution Time,Precision,Recall,F-score,N,NoiseInd,FoundNoiseInd,Time
"['Adult', 'hybridRepairFilter', 0.7, 'Asym', 100, 0]",23,9,21,47,18.4692,0.7188,0.3286,0.4510,100,"[0, 1, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 17, ...","[2, 3, 4, 10, 16, 17, 21, 23, 24, 26, 30, 34, ...",1644411471.0814


In [98]:
import pandas as pd
import sys 
sys.path.insert(0, 'scripts/')

from utils import *
names = ['Adult','DryBean','Chess','Magic','Iris',
              'ClinVarReal','ClinVarArt','Encode','RNA1','RNA2','RNA3']

names = ['Adult','DryBean','Chess','Magic','Iris','ClinVarReal','ClinVarArt','Encode']
#names = ['Adult']
nClasses = [2,6,9,2,3,3,3,2]
li = []

for i,name in enumerate(names):
    df_raw = pd.read_csv('datasets/'+name+'.csv.gz', sep = '\t',compression='zip')
    n = nClasses[i]    

    nl=1
    if name in ['ClinVarReal', 'Encode']:
        nl=2
        

    df = getProperLabels(df_raw,nl)
    df = decreaseClassNumber(df,n)
    df = getProperLabels(df,nl) #need this twice

    data = df[df.columns[~df.columns.str.contains('Label',na=False)]]
    labels = df[df.columns[df.columns.str.contains('Label',na=False)]]
    data = pd.get_dummies(data)
    data = data.join(labels)
    #data = balance(data)
    data = data.fillna(0)
  #  print(data.shape)
  #  print(data.iloc[:,-1].value_counts())
    a = df_raw.shape[0]
    b = df_raw.shape[1]-nl
    c = data.shape[1]-nl
    d = len(labels.iloc[:,-1].unique())
    e = labels.iloc[:,-1].value_counts(normalize = True).round(2).to_dict()
    li.append([name,a,d,b,c,e])
    #data = data.sample(n=min(len(data),10000))
    #data.to_csv('datasets/'+name+'.csv.gz',sep = '\t', index = None, 
     #        compression = 'zip')


In [108]:
li.append(['RNA1','','','','',''])
li.append(['RNA2','','','','',''])

li.append(['RNA3','','','','',''])

df = pd.DataFrame(li)
df.columns = ['Name','Size','Number of classes', 'Number of features','Number of dummied features'
             , 'Distibution of classes']
df['Description']=''
df.to_csv("figures/dataets.csv", sep = "\t",index = None)

